In [ ]:
from __future__ import print_function
import ipywidgets as widgets
import plotly.graph_objects as go
from epynet import Network

In [ ]:
!ls ../water_networks/

In [ ]:
wds = Network('../water_networks/anytown_master.inp')
wds.solve()

# Topology of Anytown

In [ ]:
def assemble_2d_traces(wds):
    junc_x = []
    junc_y = []
    junc_z = []
    for junc in wds.junctions:
        junc_x.append(junc.coordinates[0])
        junc_y.append(junc.coordinates[1])
        junc_z.append(junc.elevation)

    junc_trace = go.Scatter(
        x = junc_x,
        y = junc_y,
        mode='markers'
        )

    pipe_x = []
    pipe_y = []
    pipe_z = []
    for pipe in wds.pipes:
        if (pipe.from_node.index in list(wds.junctions.index)) and (pipe.to_node.index in list(wds.junctions.index)):
            pipe_x.append(pipe.from_node.coordinates[0])
            pipe_x.append(pipe.to_node.coordinates[0])
            pipe_x.append(float('nan'))

            pipe_y.append(pipe.from_node.coordinates[1])
            pipe_y.append(pipe.to_node.coordinates[1])
            pipe_y.append(float('nan'))

            pipe_z.append(pipe.from_node.elevation)
            pipe_z.append(pipe.to_node.elevation)
            pipe_z.append(float('nan'))

    pipe_trace = go.Scatter(
        x = pipe_x,
        y = pipe_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )
    return junc_trace, pipe_trace

In [ ]:
def assemble_3d_traces(wds):
    junc_x = []
    junc_y = []
    junc_z = []
    for junc in wds.junctions:
        junc_x.append(junc.coordinates[0])
        junc_y.append(junc.coordinates[1])
        junc_z.append(junc.elevation)

    junc_trace = go.Scatter3d(
        x = junc_x,
        y = junc_y,
        z = junc_z,
        mode='markers'
        )

    pipe_x = []
    pipe_y = []
    pipe_z = []
    for pipe in wds.pipes:
        if (pipe.from_node.index in list(wds.junctions.index)) and (pipe.to_node.index in list(wds.junctions.index)):
            pipe_x.append(pipe.from_node.coordinates[0])
            pipe_x.append(pipe.to_node.coordinates[0])
            pipe_x.append(float('nan'))

            pipe_y.append(pipe.from_node.coordinates[1])
            pipe_y.append(pipe.to_node.coordinates[1])
            pipe_y.append(float('nan'))

            pipe_z.append(pipe.from_node.elevation)
            pipe_z.append(pipe.to_node.elevation)
            pipe_z.append(float('nan'))

    pipe_trace = go.Scatter3d(
        x = pipe_x,
        y = pipe_y,
        z = pipe_z,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )
    return junc_trace, pipe_trace

In [ ]:
junc_trace, pipe_trace = assemble_2d_traces(wds)

junc_trace.marker = dict(
    showscale = True,
    colorscale = 'YlGnBu',
    reversescale = True,
    color = list(wds.junctions.head),
    size = 10,
    colorbar = dict(
        thickness = 15,
        title = 'Head, m',
        xanchor = 'left',
        titleside = 'right'
    ))

fig = go.FigureWidget(
    data = [pipe_trace],
    layout = go.Layout(
        title = 'Anytown',
        titlefont_size = 16,
        showlegend = False,
        hovermode = 'closest',
        margin = dict(b=20,l=5,r=5,t=40),
        xaxis = dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis = dict(showgrid=False, zeroline=False, showticklabels=False))
        )

In [ ]:
w = widgets.IntSlider(value=75)
w

In [ ]:
button = widgets.Button(description="Click Me!")

display(button)

def on_button_clicked(b):
    global fig
    wds.pumps.speed = wds.pumps.speed * w.value / 100 + .35
    wds.solve()
    junc_trace.marker.color = list(wds.junctions.head)
    #fig.add_trace(pipe_trace)
    fig.add_trace(junc_trace)
    fig.plotly_update()

button.on_click(on_button_clicked)
display(widgets.VBox([fig]))

# Bokeh solution

In [ ]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.models import ColumnDataSource as cds
output_notebook()

In [ ]:
junc_trace, pipe_trace = assemble_2d_traces(wds)

data = cds(data={
    'x': junc_trace['x'],
    'y': junc_trace['y'],
    'head': wds.junctions.head
    }
)

mapper = linear_cmap(
    field_name = 'head',
    palette = "Viridis11",
    low = min(wds.junctions.head),
    high = max(wds.junctions.head)
)

In [ ]:
p = figure()
edges = p.line(pipe_trace['x'], pipe_trace['y'])
nodes = p.circle(x='x', y='y', color=mapper, source=data, size=12)

In [ ]:
def update(speed=60):
    for pump in wds.pumps:
        pump.speed = speed / 100 + .3
    wds.solve()
    nodes.data_source.data['head'] = wds.junctions.head
    push_notebook()

In [ ]:
show(p, notebook_handle=True)

In [ ]:
interact(update, speed=(0, 100, 1))